In [101]:
from sklearn import *
import lightgbm as lgbm
import pandas as pd
import numpy as np
import re
import nltk
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
nltk.download("stopwords")
plt.style.use('ggplot')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/charubaiel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [206]:

# pos \ neg - комменты с твиттера http://study.mokoron.com/ 
# labeled каггловский датасет по токсикам https://www.kaggle.com/blackmoon/russian-language-toxic-comments
# Изначально взял только кагловский сет на 14 тыс. комментариев, но он оказался слишком специфичным
# Проверить в ручную эти модели можно загрузив дампы из models
# модели без номеров - обучены только на Kaggle датасете
# модели с номером 2 - обучены на смешенном датасете
# При обучении только на нем, модели выдавали под 83+% метрик, при чем не важно как делился сет, через кроссвалидацию от 2 - 20 цифры успешности оказывались слишком большими.
# При ручном тестировании модель жутко косило в сторону токсичности, любые комменты она в большей степени относила к токсичным.
# Для решения проблемы - решил достать нейтральные сеты схожей тематики и добавить их в виде "не токсичных"
# Таким образом решая сразу и проблему объема выборки и однородности \ плохой генерализации выборки

In [103]:
twitter_neg = pd.read_csv('data/negative.csv',sep=';',header=None,usecols=[3])
twitter_pos = pd.read_csv('data/positive.csv',sep=';',header=None,usecols=[3])
vk_all = pd.read_csv('data/labeled.csv')

In [104]:
ttl_toxic = vk_all.append(twitter_pos.rename(columns={3:'comment'}).sample(5000)).fillna(0)


In [105]:

# Распределение специально формировалось не сильно дизбалансным, чтобы не решать отдельно проблему дисбаланса
# В целом такой объем выборки сформировался эксперементальным путем

In [106]:
ttl_toxic['toxic'].value_counts(normalize=True)

0.0    0.751391
1.0    0.248609
Name: toxic, dtype: float64

In [107]:
scores = {}

In [108]:
data,val_data,target,val_target = model_selection.train_test_split(ttl_toxic['comment'],ttl_toxic['toxic'],train_size=.75,stratify=ttl_toxic['toxic'])

### stupid baseline

In [109]:
# Первый этап - понять наш бейслайн, от чего мы может отталкиваться и какие в целом данные у нас есть.
# Т.к. в наборе по большому счету только текст, без дополнительных признаков людей, EDA с генерацией фичей поможет очень слабо
# По хорошему для формирования достаточно хороших прогнозов на сложных структурах имеет смысл использовать нейросети
# Т.к. у нейросетей не маленький инференс + с нейросетями я знаком поверхностно для мвп и понятия возможностей начал с простого мл

In [110]:
tf = feature_extraction.text.TfidfVectorizer()

In [111]:
tf.fit(data)

TfidfVectorizer()

In [112]:
# Для удобства использования, упрощения деплоя ,снижения захламленности и своеобразного инкапсулирования методов - все модельки сразу идут в пайплайны
# Почти во всех моделях для упролщения работы добавлял распределение классов

In [113]:
clf_base = linear_model.LogisticRegression(max_iter=1000,class_weight=target.value_counts(normalize=True).to_dict())
pipe_base = pipeline.make_pipeline(tf,clf_base)

In [114]:
# Проверка проходит специально без какой то оптимизации гиперпараметров чтобы понять сырые возможности

In [115]:
scores['baseline'] = pd.DataFrame(model_selection.cross_validate(pipe_base,data,target,scoring=['f1','precision','recall'],cv=5,verbose=10)).mean()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] START .....................................................................
[CV] END  f1: (test=0.695) precision: (test=0.764) recall: (test=0.638) total time=  12.6s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.6s remaining:    0.0s


[CV] END  f1: (test=0.669) precision: (test=0.743) recall: (test=0.608) total time=  12.7s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.3s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   34.8s remaining:    0.0s


[CV] END  f1: (test=0.691) precision: (test=0.738) recall: (test=0.649) total time=  13.7s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   48.5s remaining:    0.0s


[CV] END  f1: (test=0.682) precision: (test=0.756) recall: (test=0.621) total time=  14.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.1min finished


In [116]:
# Для сравнения все CSV выборки пишу в словарь для конечной оценки методов

In [117]:
pd.DataFrame(scores)

,baseline
fit_time,12.519123
score_time,0.099639
test_f1,0.689250
test_precision,0.755897
test_recall,0.633597


In [118]:
pipe_base.fit(data,target)

/home/charubaiel/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(C=6, class_weight={0.0: 10940, 1.0: 3619},
                                    max_iter=1000))])

### work with text

In [119]:
# Первое что необходимо сделать - нормализовать текст, чтобы уменьшить кол-ва шума в данных
# Наташа хоть и выглядит страшнее, но быстрее аналогов на порядок + имеет встроенные не плохие ембединги

In [120]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
stopwords = nltk.corpus.stopwords.words('russian')

In [121]:
def normalizer (text):
    words_only = re.sub('[^А-я]+',' ',text.lower())
    doc = Doc(words_only)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    clean_text = []
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        if (token.lemma not in stopwords) & (len(set(token.lemma))>1):
            clean_text.append(token.lemma)
            
    return ' '.join(clean_text)

In [122]:
# Обрабатывать отдельно каждое предложение затратно за счет ресурсов на перепарсинг
# Намного эффективнее подать на вход большой кусок текста, а потом расформировать его нобратно по комментам

In [123]:
raw_data = data.copy()
old_text = ' жожо '.join(data)
new_text = normalizer(old_text)
data = pd.Series(new_text.split('жожо'))
raw_val_data = val_data.copy()
val_data = pd.Series(normalizer(' жожо '.join(val_data)).split('жожо'))

In [124]:
clf_norm = linear_model.LogisticRegression(max_iter=1000,class_weight= target.value_counts(normalize=True).to_dict())
pipe_norm = pipeline.make_pipeline(tf,clf_norm)
scores['normalize'] = pd.DataFrame(model_selection.cross_validate(pipe_norm,data,target,scoring=['f1','precision','recall'],cv=5)).mean()

In [125]:
pipe_norm.fit(data,target)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression',
                 LogisticRegression(class_weight={0.0: 0.7514252352496738,
                                                  1.0: 0.24857476475032625},
                                    max_iter=1000))])

In [126]:
# Видны сразу изменения в балансе метрик и во времени обучения модели
# Скорее всего такой буст бейслайну дал факт того, что выборки глобально отличаются и он просто нашел различия в стилистике между датасетами

In [127]:
pd.DataFrame(scores)

,baseline,normalize
fit_time,12.519123,0.315521
score_time,0.099639,0.046333
test_f1,0.689250,0.097987
test_precision,0.755897,0.983043
test_recall,0.633597,0.051677


### micro EDA

In [128]:
eda = vk_all.copy(deep=True)

In [129]:
# Попытка посмотреть на наши данные

In [130]:
eda['txt_len'] = eda['comment'].str.len()
eda['txt_len_avg'] = eda['comment'].str.split().apply(lambda x: np.mean([len(word) for word in x]))
eda['txt_words'] = eda['comment'].str.count(' ')
eda['txt_puncts'] = eda['comment'].str.count('[^\w^ ]')
eda['txt_upper_cnt'] = eda['comment'].apply(lambda x: len([i for i in x if i.isupper()]))
eda['txt_pct_upper'] = eda['txt_upper_cnt'] / eda['txt_len']
eda['txt_pos_punc'] = eda['comment'].apply(lambda text: len(re.findall(r'\)|D',text)))
eda['txt_neg_punc'] = eda['comment'].apply(lambda text: len(re.findall(r'\(|C|c|С|c',text)))

In [131]:
eda.groupby('toxic')[eda.filter(regex='txt').columns].agg(['mean','std'])

txt_len             txt_len_avg            txt_words             \
             mean         std        mean       std       mean        std   
toxic                                                                       
0.0    194.213332  274.750067    5.312308  1.063207  29.713436  40.811415   
1.0    141.392665  261.776417    5.467866  1.466984  21.449233  42.106635   

      txt_puncts           txt_upper_cnt            txt_pct_upper            \
            mean       std          mean        std          mean       std   
toxic                                                                         
0.0     7.481849  9.135183      3.984978   7.812980      0.023631  0.027502   
1.0     5.972234  9.188326      6.595939  26.932055      0.052774  0.130923   

      txt_pos_punc           txt_neg_punc            
              mean       std         mean       std  
toxic                                                
0.0       0.279261  0.709825     0.499478  1.353467  
1.0       0.094488  0.434023     0.454414  1.689128

In [132]:
eda.iloc[:,1:].corr()

,toxic,txt_len,txt_len_avg,txt_words,txt_puncts,txt_upper_cnt,txt_pct_upper,txt_pos_punc,txt_neg_punc
toxic,1.000000,-0.091782,0.060394,-0.094138,-0.077608,0.072997,0.171514,-0.136895,-0.014424
txt_len,-0.091782,1.000000,0.024703,0.991756,0.924170,0.370052,-0.047513,0.382291,0.529863
txt_len_avg,0.060394,0.024703,1.000000,-0.025150,0.016616,0.022158,0.079450,-0.001665,0.037176
txt_words,-0.094138,0.991756,-0.025150,1.000000,0.929034,0.368631,-0.049015,0.378389,0.514993
txt_puncts,-0.077608,0.924170,0.016616,0.929034,1.000000,0.346096,-0.033393,0.418016,0.497885
txt_upper_cnt,0.072997,0.370052,0.022158,0.368631,0.346096,1.000000,0.565105,0.115176,0.692265
txt_pct_upper,0.171514,-0.047513,0.079450,-0.049015,-0.033393,0.565105,1.000000,-0.040094,0.312005
txt_pos_punc,-0.136895,0.382291,-0.001665,0.378389,0.418016,0.115176,-0.040094,1.000000,0.395420
txt_neg_punc,-0.014424,0.529863,0.037176,0.514993,0.497885,0.692265,0.312005,0.395420,1.000000


### features

In [133]:
# Вариант с дорбавлениями фичей для модели, исходя из EDA есть слабый шанс на улучшение, без особой потери в скорости \ инференсе

In [134]:
def features_from_text(df):
    df = pd.DataFrame(df)
    df.columns=['text']
    df['txt_len'] = df['text'].str.len()
    df['txt_len_avg'] = df['text'].str.split().apply(lambda x: np.mean([len(word) for word in x]))
    df['txt_words'] = df['text'].str.count(' ')
    df['txt_puncts'] = df['text'].str.count('[^\w^ ]')
    df['txt_upper_cnt'] = df['text'].apply(lambda x: len([i for i in x if i.isupper()]))
    df['txt_pos_punc'] = df['text'].apply(lambda text: len(re.findall(r'\)|D',text)))
    df['txt_neg_punc'] = df['text'].apply(lambda text: len(re.findall(r'\(|C|c|С|c',text)))
    
    return df.drop('text',axis=1).fillna(0)

In [135]:
fe_tf = feature_extraction.text.TfidfVectorizer()
fe_tf.fit(data,target)
new_fe=preprocessing.FunctionTransformer(features_from_text)
text_preproc = pipeline.FeatureUnion([('idf',fe_tf),('fe',new_fe)])

In [136]:
clf_fe = linear_model.LogisticRegression(max_iter=1000,class_weight= target.value_counts(normalize=True).to_dict())
pipe_fe = pipeline.Pipeline([('preproc',text_preproc),('clf',clf_fe)])

In [137]:
scores['features'] = pd.DataFrame(model_selection.cross_validate(pipe_fe,data,target,scoring=['f1','precision','recall'],cv=5)).mean()

/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._

In [138]:
pipe_fe.fit(data,target)

/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Pipeline(steps=[('preproc',
                 FeatureUnion(transformer_list=[('idf', TfidfVectorizer()),
                                                ('fe',
                                                 FunctionTransformer(func=<function features_from_text at 0x7fb4f9918dc0>))])),
                ('clf',
                 LogisticRegression(class_weight={0.0: 0.7514252352496738,
                                                  1.0: 0.24857476475032625},
                                    max_iter=1000))])

In [139]:
pd.DataFrame(scores)

,baseline,normalize,features
fit_time,12.519123,0.315521,1.081329
score_time,0.099639,0.046333,0.143306
test_f1,0.689250,0.097987,0.103340
test_precision,0.755897,0.983043,0.966164
test_recall,0.633597,0.051677,0.054716


### vectors

In [140]:
# Готовые векторные пространства имеют больший диапазон слов, за счет чего могут быть эффективнее преобразования локальных сетов данных
# + они сформированны на более сложных алгоритмах, что дает бОльшую генерализующую и контекстную силу
# конкретно навекрвские вектора весят не много, и дают сравнимый эффект с крутыми собратьями
# https://github.com/natasha/navec

In [141]:
from navec import Navec

In [142]:
nav = Navec.load('models/emb_navec.tar')

In [143]:
def get_sentence_vector(sentence_list):
    vectors = []
    for sentence in sentence_list:
        sent_vec = []
        for i in sentence.split():
            if i in nav:
                sent_vec.append(nav[i])
            else:
                sent_vec.append(nav['<unk>'])
        if sentence.strip() == '':
            sent_vec = [nav['<unk>']]
        vectors.append(np.mean(sent_vec,axis=0))
    return np.vstack(vectors)

In [144]:
vec_func = preprocessing.FunctionTransformer(get_sentence_vector)

In [145]:
clf_vec = linear_model.LogisticRegression(max_iter=1000,class_weight= target.value_counts(normalize=True).to_dict())
pipe_vec = pipeline.make_pipeline(vec_func,clf_vec)

In [146]:
scores['sample_vec'] = pd.DataFrame(model_selection.cross_validate(pipe_vec,data,target,scoring=['f1','precision','recall'],cv=5)).mean()

In [147]:
pipe_vec.fit(data,target)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function get_sentence_vector at 0x7fb4e016c790>)),
                ('logisticregression',
                 LogisticRegression(class_weight={0.0: 0.7514252352496738,
                                                  1.0: 0.24857476475032625},
                                    max_iter=1000))])

In [148]:
pd.DataFrame(scores)

,baseline,normalize,features,sample_vec
fit_time,12.519123,0.315521,1.081329,1.405890
score_time,0.099639,0.046333,0.143306,0.309307
test_f1,0.689250,0.097987,0.103340,0.456973
test_precision,0.755897,0.983043,0.966164,0.833795
test_recall,0.633597,0.051677,0.054716,0.315007


### combo

In [149]:
# Один из способов собрать по дешевке fasttext на коленях использовать для векторизации отдельно слова, и отдельно составные их части от 2-5 букв
# в теории можно найти частоиспользуемые корневые вещи по типу "бл", "еб" от мата и тд.

In [150]:
word_vectorizer = feature_extraction.text.TfidfVectorizer()
word_vectorizer.fit(data)

TfidfVectorizer()

In [151]:
char_vectorizer = feature_extraction.text.TfidfVectorizer(
    min_df=3,
    sublinear_tf=True,
    analyzer='char',
    ngram_range=(2,4))
char_vectorizer.fit(data)

TfidfVectorizer(analyzer='char', min_df=3, ngram_range=(2, 4),
                sublinear_tf=True)

In [152]:
idf_fu = pipeline.FeatureUnion([('idf_w',word_vectorizer),('idf_c',char_vectorizer)])

In [153]:
clf_2idf = linear_model.LogisticRegression(max_iter=1000,C=6,class_weight= target.value_counts(normalize=True).to_dict())
pipe_idf_fe = pipeline.Pipeline([('idf',idf_fu),('clf',clf_2idf)])

In [154]:
scores['idf_features'] = pd.DataFrame(model_selection.cross_validate(pipe_idf_fe,data,target,scoring=['f1','precision','recall'],cv=5)).mean()

In [155]:
pipe_idf_fe.fit(data,target)

Pipeline(steps=[('idf',
                 FeatureUnion(transformer_list=[('idf_w', TfidfVectorizer()),
                                                ('idf_c',
                                                 TfidfVectorizer(analyzer='char',
                                                                 min_df=3,
                                                                 ngram_range=(2,
                                                                              4),
                                                                 sublinear_tf=True))])),
                ('clf',
                 LogisticRegression(C=6,
                                    class_weight={0.0: 0.7514252352496738,
                                                  1.0: 0.24857476475032625},
                                    max_iter=1000))])

In [156]:
pd.DataFrame(scores)

,baseline,normalize,features,sample_vec,idf_features
fit_time,12.519123,0.315521,1.081329,1.405890,4.759447
score_time,0.099639,0.046333,0.143306,0.309307,0.482817
test_f1,0.689250,0.097987,0.103340,0.456973,0.625519
test_precision,0.755897,0.983043,0.966164,0.833795,0.930100
test_recall,0.633597,0.051677,0.054716,0.315007,0.471409


### blend models

In [157]:
# Для генерализации ответов, один из вариантов это блендинг моделей разного рода
# Тестирую разнородные модели, потом из них беру только те, которые показали сравнительно не плохой результат
# Или выделились по конкретным метрикам

In [158]:
vec_func = preprocessing.FunctionTransformer(get_sentence_vector)

In [159]:
class_prior=target.value_counts(normalize=True).values[::-1]

In [160]:
clf_lr_vec = linear_model.LogisticRegression(max_iter=1000,C=6,class_weight= target.value_counts().to_dict())
clf_nb_vec = naive_bayes.BernoulliNB(class_prior=class_prior)
clf_knn_vec = neighbors.KNeighborsClassifier(30)
clf_svc_vec = linear_model.SGDClassifier(loss='modified_huber',class_weight= target.value_counts().to_dict())
clf_rf_vec = lgbm.LGBMClassifier(n_estimators=1500,learning_rate=0.07,num_leaves=15,class_weight= target.value_counts().to_dict())
clf_mlp_vec = neural_network.MLPClassifier(hidden_layer_sizes=(300,1),max_iter=1000,learning_rate='adaptive')


In [161]:
models = {}
for n,model in tqdm(enumerate([clf_lr_vec,clf_nb_vec,clf_knn_vec,clf_svc_vec,clf_rf_vec,clf_mlp_vec])):
    tmp_pipe = pipeline.make_pipeline(vec_func,model)
    models[n] = pd.DataFrame(model_selection.cross_validate(tmp_pipe,data,target,scoring=['f1','precision','recall'],cv=5)).mean()
    print(models[n])

1it [00:10, 10.32s/it]

fit_time          1.732373
score_time        0.330093
test_f1           0.483680
test_precision    0.813432
test_recall       0.344575
dtype: float64


2it [00:17,  8.75s/it]

fit_time          1.207198
score_time        0.319870
test_f1           0.597102
test_precision    0.521148
test_recall       0.699088
dtype: float64


3it [00:28,  9.38s/it]

...

fit_time          2.056230
score_time        0.304323
test_f1           0.485741
test_precision    0.690036
test_recall       0.398731
dtype: float64


5it [01:19, 20.75s/it]

fit_time          7.515308
score_time        0.327282
test_f1           0.672619
test_precision    0.818714
test_recall       0.570883
dtype: float64


6it [02:26, 24.45s/it]

fit_time          13.207715
score_time         0.308060
test_f1            0.706715
test_precision     0.735303
test_recall        0.682252
dtype: float64


In [162]:
pd.DataFrame(models).rename(columns = {0:'lr',1:'nb',2:'knn',3:'svc',4:'gbm',5:'mlp'}).T.sort_values(by='test_f1',ascending=False)

,fit_time,score_time,test_f1,test_precision,test_recall
mlp,13.207715,0.308060,0.706715,0.735303,0.682252
gbm,7.515308,0.327282,0.672619,0.818714,0.570883
knn,1.197858,0.826378,0.657408,0.596258,0.732791
nb,1.207198,0.319870,0.597102,0.521148,0.699088
svc,2.056230,0.304323,0.485741,0.690036,0.398731
lr,1.732373,0.330093,0.483680,0.813432,0.344575


In [163]:
# Лучшие собрал в классифаер, который выбирает самый "уверенный" ответ

In [164]:

vote_vec = ensemble.VotingClassifier(estimators=[('knn',clf_knn_vec),('gbm',clf_rf_vec),('mlp',clf_mlp_vec)],voting='soft')

In [165]:
pipe_vote_vec = pipeline.make_pipeline(vec_func,vote_vec)

In [166]:
scores['vote_models'] = pd.DataFrame(model_selection.cross_validate(pipe_vote_vec,data,target,scoring=['f1','precision','recall'],cv=5,n_jobs=4)).mean()

In [167]:
pipe_vote_vec.fit(data,target)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function get_sentence_vector at 0x7fb4e016c790>)),
                ('votingclassifier',
                 VotingClassifier(estimators=[('knn',
                                               KNeighborsClassifier(n_neighbors=30)),
                                              ('gbm',
                                               LGBMClassifier(class_weight={0.0: 10940,
                                                                            1.0: 3619},
                                                              learning_rate=0.07,
                                                              n_estimators=1500,
                                                              num_leaves=15)),
                                              ('mlp',
                                               MLPClassifier(hidden_layer_sizes=(300,
                                                                             

In [168]:
pd.DataFrame(scores)

,baseline,normalize,features,sample_vec,idf_features,vote_models
fit_time,12.519123,0.315521,1.081329,1.405890,4.759447,38.413581
score_time,0.099639,0.046333,0.143306,0.309307,0.482817,1.240588
test_f1,0.689250,0.097987,0.103340,0.456973,0.625519,0.700482
test_precision,0.755897,0.983043,0.966164,0.833795,0.930100,0.784968
test_recall,0.633597,0.051677,0.054716,0.315007,0.471409,0.634994


### blend results

In [169]:
# До до этого блендили модели, еще один вариант сблендить подходы к моделированию
# При объединении векторного подхода + собвственных idf ных , результат должен быть точнее

In [177]:

def voting(sentences,func=np.max):
    sentences = [normalizer(txt) for txt in sentences]
    probs = []
    probs.append(pipe_idf_fe.predict_proba(sentences)[:,1])
    probs.append(pipe_vote_vec.predict_proba(sentences)[:,1])
    probs.append(pipe_vec.predict_proba(sentences)[:,1])
    return np.apply_over_axes(func,np.array(probs),axes=0)[0]

In [200]:
eval_data = defaultdict(list)
for model in [pipe_norm,pipe_fe,pipe_idf_fe,pipe_vec,pipe_vote_vec]:
    eval_data['f1'].append(metrics.f1_score(val_target,model.predict(val_data)))
    eval_data['recall'].append(metrics.recall_score(val_target,model.predict(val_data)))
    eval_data['precision'].append(metrics.precision_score(val_target,model.predict(val_data)))
    eval_data['roc_auc'].append(metrics.roc_auc_score(val_target,model.predict(val_data)))

/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/charubaiel/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [201]:
eval_data['f1'].append(metrics.f1_score(val_target,voting(val_data).round()))
eval_data['recall'].append(metrics.recall_score(val_target,voting(val_data).round()))
eval_data['precision'].append(metrics.precision_score(val_target,voting(val_data).round()))
eval_data['roc_auc'].append(metrics.roc_auc_score(val_target,voting(val_data).round()))

In [202]:
eval_data['f1'].append(metrics.f1_score(val_target,voting(val_data,func=np.mean).round()))
eval_data['recall'].append(metrics.recall_score(val_target,voting(val_data,func=np.mean).round()))
eval_data['precision'].append(metrics.precision_score(val_target,voting(val_data,func=np.mean).round()))
eval_data['roc_auc'].append(metrics.roc_auc_score(val_target,voting(val_data,func=np.mean).round()))

In [203]:
# Итог радует
# Хоть и не получили супргероя который выполняет сразу обе задачи ( и полноту и сложность)
# Но зато есть несколько вариантов для выбора исходя из реальной задачи (инференс \ точность \ полнота)
# Если говорить о метриках, то 

In [205]:
pd.DataFrame(eval_data,index=['norm','fe','idf_df','vec','vote_vec','blend_max','blend_mean']).sort_values(by='f1',ascending=False)

,f1,recall,precision,roc_auc
blend_max,0.740933,0.710853,0.773670,0.821005
vote_vec,0.701975,0.647887,0.765916,0.791168
blend_mean,0.636990,0.497929,0.883824,0.738131
idf_df,0.625753,0.473074,0.923948,0.730091
vec,0.445921,0.305717,0.823661,0.642025
fe,0.102041,0.053853,0.970149,0.526652
norm,0.086957,0.045568,0.948276,0.522372


In [ ]:
# Это проходные результаты, которы можно еще доработать \ проверить \ затюнить

### testing

In [192]:

def check_text (text):
    return pd.Series({'pure_idf_model' : pipe_norm.predict_proba([text])[:,1][0],
    'some_features_model' : pipe_fe.predict_proba([text])[:,1][0],
    'double_idf_model' :pipe_idf_fe.predict_proba([text])[:,1][0],
    'pure_vectors_model' :pipe_vec.predict_proba([text])[:,1][0],
    'blending_models' : pipe_vote_vec.predict_proba([text])[:,1][0],
    'blending_ttl_mean' : voting([text],func=np.mean)[0],
    'blending_ttl_max' : voting([text],func=np.max)[0]},name='Степень токсичности')

In [196]:
def time_predict(model,text):
    try:
        return model.predict_proba([text])[:,1][0]
    except:
        return model([text])

In [185]:
# Смотрим на инференс и решаем что будем использовать
# Так же параллельно руками проверяем работоспособность

In [186]:
%%timeit
time_predict(pipe_norm,'Началось все с заселения! Минут 20 искали бронь - с трудом нашли. В документах ошибка на ошибке, пришлось несколько раз переделывать. Девушка на ресепшене с трудом с помощью калькулятора смогла посчитать сколько надо сдачу дать, но при этом все равно умудрилась обмануть и при этом с наивным взглядом, протягивая эти 100 рублей выдавила из себя: «ну я же не обманула». ')

562 µs ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [187]:
%%timeit
time_predict(pipe_fe,'Началось все с заселения! Минут 20 искали бронь - с трудом нашли. В документах ошибка на ошибке, пришлось несколько раз переделывать. Девушка на ресепшене с трудом с помощью калькулятора смогла посчитать сколько надо сдачу дать, но при этом все равно умудрилась обмануть и при этом с наивным взглядом, протягивая эти 100 рублей выдавила из себя: «ну я же не обманула». ')

4.73 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [188]:
%%timeit
time_predict(pipe_idf_fe,'Началось все с заселения! Минут 20 искали бронь - с трудом нашли. В документах ошибка на ошибке, пришлось несколько раз переделывать. Девушка на ресепшене с трудом с помощью калькулятора смогла посчитать сколько надо сдачу дать, но при этом все равно умудрилась обмануть и при этом с наивным взглядом, протягивая эти 100 рублей выдавила из себя: «ну я же не обманула». ')

2.21 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [189]:
%%timeit
time_predict(pipe_vec,'Началось все с заселения! Минут 20 искали бронь - с трудом нашли. В документах ошибка на ошибке, пришлось несколько раз переделывать. Девушка на ресепшене с трудом с помощью калькулятора смогла посчитать сколько надо сдачу дать, но при этом все равно умудрилась обмануть и при этом с наивным взглядом, протягивая эти 100 рублей выдавила из себя: «ну я же не обманула». ')

460 µs ± 2.49 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [190]:
%%timeit
time_predict(pipe_vote_vec,'Началось все с заселения! Минут 20 искали бронь - с трудом нашли. В документах ошибка на ошибке, пришлось несколько раз переделывать. Девушка на ресепшене с трудом с помощью калькулятора смогла посчитать сколько надо сдачу дать, но при этом все равно умудрилась обмануть и при этом с наивным взглядом, протягивая эти 100 рублей выдавила из себя: «ну я же не обманула». ')

28.5 ms ± 2.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [197]:
%%timeit
time_predict(voting,'Началось все с заселения! Минут 20 искали бронь - с трудом нашли. В документах ошибка на ошибке, пришлось несколько раз переделывать. Девушка на ресепшене с трудом с помощью калькулятора смогла посчитать сколько надо сдачу дать, но при этом все равно умудрилась обмануть и при этом с наивным взглядом, протягивая эти 100 рублей выдавила из себя: «ну я же не обманула». ')

35.7 ms ± 3.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [198]:
%%timeit
check_text('Началось все с заселения! Минут 20 искали бронь - с трудом нашли. В документах ошибка на ошибке, пришлось несколько раз переделывать. Девушка на ресепшене с трудом с помощью калькулятора смогла посчитать сколько надо сдачу дать, но при этом все равно умудрилась обмануть и при этом с наивным взглядом, протягивая эти 100 рублей выдавила из себя: «ну я же не обманула». ')

111 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [199]:
check_text('Эх, жалко что ты такой безнадежный').to_frame().style.format('{:.2%}')

,Степень токсичности
pure_idf_model,13.45%
some_features_model,13.04%
double_idf_model,17.59%
pure_vectors_model,51.31%
blending_models,68.27%
blending_ttl_mean,54.06%
blending_ttl_max,81.60%


### final

In [ ]:
import joblib

In [ ]:
joblib.dump(pipe_vec,'models/model_vec_2.joblib')
joblib.dump(pipe_idf_fe,'models/model_idf_2.joblib')
joblib.dump(pipe_vote_vec,'models/model_vec_vote_2.joblib')